In [14]:
import boto3 
import s3fs
import pandas as pd
import datetime
from tzlocal import get_localzone as tzlocal
from random import randint
from  datetime import timedelta
import schedule 
import time

bucket_name = 'fog-datasets'
s3_con = boto3.client('s3')

file_name = 'streaming-data-mock.csv'
data = []
file_str = s3_con.get_object(Bucket=bucket_name, Key=file_name).get('Body').read().decode('utf-8')
df = pd.read_csv('s3:{}/{}'.format(bucket_name,file_name),index_col=[0])
df.index = pd.to_datetime(df.index)



frequency_min = 15


def job():
    print("starting new job...")
    today = datetime.datetime.today()
    end_time_relative = today.replace(year=2019, month=1, day = 7 + today.weekday() )
    start_time_relative = end_time_relative - timedelta(minutes=frequency_min)
    df_to_send = df[df.index < end_time_relative]
    df_to_send = df_to_send[df_to_send.index > start_time_relative] 
    df_to_send.tail()

    datapoints = []
    for time, row in df_to_send.iterrows():
        datapoints.append({'Timestamp':datetime.datetime(time.year,time.month,time.day, time.hour, time.minute, tzinfo=None), 'SampleCount': row.SampleCount, 'Unit': 'None'})

    if(len(datapoints) ==0):
        print("No datapoints! skipping upload...")
        return

    # Upload preprocessed data for deep AR
    data_bucket_name = 'fog-datasets'
    directory = 'streaming-data-mock'
    e_t = today - timedelta(minutes=frequency_min)
    filename = "{}-{}-{}-{}:{}:::{}-{}-{}-{}:{}.json".format(e_t.year,e_t.month,e_t.day,e_t.hour,e_t.minute,today.year,today.month,today.day,today.hour,today.minute)

    print ("Appending {} datapoints to {}".format(len(datapoints), filename))
    s3_data_path = "{}/{}/{}".format(data_bucket_name, directory, filename)
    s3filesystem = s3fs.S3FileSystem()

    out = {
        'response-code-200': {'Label': 'response-code-200', 'Datapoints':datapoints}
    }
    try:
        with s3filesystem.open(s3_data_path, 'w') as fp:
            fp.write(repr(out))
    except:
        print("Exception!")
        
        
schedule.every(frequency_min).minutes.do(job)
while True:
    schedule.run_pending()
    time.sleep(30)


KeyboardInterrupt: 

In [6]:
 file_str = s3_con.get_object(Bucket="fog-datasets", Key="streaming-data-mock/2019-2-6-12:17:::2019-2-6-12:32.json").get('Body').read().decode('utf-8')

In [9]:
print(file_str)

{'response-code-200': {'Label': 'response-code-200', 'Datapoints': [{'Timestamp': datetime.datetime(2019, 1, 9, 12, 22, tzinfo=<UTC>), 'SampleCount': 141978.0, 'Unit': 'None'}, {'Timestamp': datetime.datetime(2019, 1, 9, 12, 27, tzinfo=<UTC>), 'SampleCount': 143389.0, 'Unit': 'None'}, {'Timestamp': datetime.datetime(2019, 1, 9, 12, 32, tzinfo=<UTC>), 'SampleCount': 142713.0, 'Unit': 'None'}]}}
